In [ ]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

#These are DDP specific packages. 
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

#These are AMP (Mixed Precision) specific packages 
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [ ]:
#Install Weights and Biases package 
!pip install wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
# 1. Start a W&B run
wandb.init(project='hw9', entity='jjohns')

In [ ]:
#Set random seed so we get consistent results. 
SEED=1

In [ ]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [ ]:
START_EPOCH = 0

### Set the architecture to resnet 18 below

In [ ]:
##########################
#Set torch architecture to resnet18
#ResNet-18 is a convolutional neural network that is 18 layers deep. 
#You can load a pretrained version of the network trained on more than a million images from the ImageNet database [1]. 
#The pretrained network can classify images into 1000 object categories, such as keyboard, mouse, pencil, and many animals. 
#As a result, the network has learned rich feature representations for a wide range of images. 
#The network has an image input size of 224-by-224. 

#See here for original paper which provides implementations research team used: https://arxiv.org/pdf/1512.03385.pdf
ARCH  = 'resnet18'
# please look up how to do that
########################
#Set our hyperparams 
#Outer training loop input: How many passes through entire training set we make. If batch size = training size, then epoch = iterations. 
EPOCHS = 5
#Optimizer input: how much to change the model in response to the estimated error each time the model weights are updated
LR = 0.1
#Optimizer input: momentum is method which helps accelerate gradients vectors in the right directions, thus leading to faster convergence to local/global minima. 
MOMENTUM = 0.9
#Optimizer input: Weight decay is a regularization technique by adding a small penalty, usually the L2 norm of the weights (all the weights of the model), to the loss function. 
#loss = loss + weight decay parameter * L2 norm of the weights. 
WEIGHT_DECAY = 0.0001
PRINT_FREQ = 256
#Batch size for training / validation loss calcs. 
TRAIN_BATCH=256
VAL_BATCH=256
#Setting the argument num_workers as a positive integer will turn on multi-process data loading with the specified number of loader worker processes.
#For CUDA: we recommend using automatic memory pinning (i.e., setting pin_memory=True), which enables fast data transfer to CUDA-enabled GPUs.
WORKERS=4

#TODO: Ensure these tie to the VM directories (volume mount for Docker container)
TRAINDIR ="/workspace/storage/train"
VALDIR = "/workspace/storage/val"

In [ ]:
#Instantiate wandb with desired fiels 
wandb.init(config={"epochs": EPOCHS, "batch_size": TRAIN_BATCH, "momentum": MOMENTUM, "WEIGHT_DECAY": WEIGHT_DECAY, "arch": ARCH})

In [ ]:
# 2. Save model inputs and hyperparameters
#config = wandb.config
#config.learning_rate = LR

### Check if cuda is available here

In [ ]:
# check if cuda is available in this cell
# if it is not available, you should not go forward!
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [ ]:
WORLD_SIZE = 2
BACKEND = 'nccl'
#Assuming this should be tcp://publicipv4:inboundport
URL = 'tcp://3.18.105.108:1234'

In [ ]:
RANK = 0

In [ ]:
dist.init_process_group(backend = BACKEND, init_method = URL, world_size = WORLD_SIZE, rank = RANK)

### Assign your GPU below

In [ ]:
# Assign your GPU in this cell(they are zero indexed) (Must match RANK)
GPU = 0

In [ ]:
# set your active device to your GPU in this cell
torch.cuda.set_device(RANK)

In [ ]:
# enable algorithm optimization
#benchmark mode is good whenever your input sizes for your network do not vary. This way, cudnn will look for the optimal set of algorithms for that particular configuration (hardware + input) (which takes some time). This usually leads to faster runtime.
#But if your input sizes changes at each iteration, then cudnn will benchmark every time a new size appears, possibly leading to worse runtime performances.
cudnn.benchmark = True

### Fill in the heart of the train section below

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch):
    #AverageMeter: Computes and stores the average and current value.
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    #Show ProgressMeter
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))
    
    #Instantiate GradSCaler() for AMP
    scaler = GradScaler()

    ######################
    # switch model to train mode here
    model.train()
    ################

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #####################
        # send the images to cuda device
        # send the target to cuda device
        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)
        
        #Compute output with quantization (AMP)
        with autocast(): 
            output = model(images)
            loss = criterion(output, target)


        # compute output
        #output = model(images)
        # output = model ?? images
        

        # compute loss 
        # loss = criterion, output, target
        #loss = criterion(output, target)


        # measure accuracy and record loss
        #topk accuracy counts a model as being accurate if the correct result is in the top n predicted class probabilities (i.e. may not be the prediction)
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        #Use AverageMeter().update to update the averages 
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        #First need to zero out gradient to prevent "exploding gradient" problem. 
        #Default is to sum or accumulate losses on backward passes (loss.backward()) bc it's convient for RNN. 
        optimizer.zero_grad()
        #Backprop: Computes gradient (dloss/dx) for every parameter x which has requires_grad = True. These are accumulated into x.grad for every parameter x. 
        #x.grad += dloss/dx   -->  x += -lr * x.grad
        loss.backward()
        #Update paremeters (weights) based on previous step
        optimizer.step()
    
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})

        if i % PRINT_FREQ == 0:
            progress.display(i)

#### Fill in the validate section below

In [ ]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            
            ### send the images and target to cuda
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)


            # compute output
            # output = model ??? images?
            output = model(images)

            # compute loss
            # loss  = criterion ?? output ?? target
            loss = criterion(output, target)


            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
        
    wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})
    return top1.avg

### Save the checkpoint

In [ ]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    # save the model state!
    # state ??? 
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [ ]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [ ]:
# if we are adjusting the LR manually use this
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    #Adjusting learning drop from 30 to 20 
    lr = LR * (0.1 ** (epoch // 20))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]
#Adjusting resnet normalizations here
resnet_mean_RGB = [0.485, 0.456, 0.406]
resnet_std_RGB = [0.229, 0.224, 0.225]

In [ ]:
#Adjusting transforms with updated normalizations here (for Resnet18)
#Link: https://pytorch.org/vision/stable/models.html
normalize = transforms.Normalize(mean=resnet_mean_RGB, std=resnet_std_RGB)

In [ ]:
#TODO: Determine which IMG_SIZE we need
#IMG_SIZE = 32
IMG_SIZE = 224

### Initialize the model using the architecture you selected above

In [ ]:
# select the model
# model = ... 
#TODO: Determine how many classes we need 
NUM_CLASSES = 1000

model = models.__dict__[ARCH]()

model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

In [ ]:
# send the model to the cuda device.. 
model.cuda(GPU)

In [ ]:
#Wrap model in DDP class for distributed processing across 2 VMs
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])

### Instantiate the loss to cross entropy

In [ ]:
# use the cross-entropy loss
criterion = nn.CrossEntropyLoss().cuda(GPU)

### Instantiate the optimizer to SGD

In [ ]:
# use SGD .. use the momentum and weight decay vars
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

#### Create the learning rate scheduler

In [ ]:
# https://pytorch.org/docs/stable/optim.html
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
#See here for these steps: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
transform_train = transforms.Compose([
    #Adjusting crop call and size
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(resnet_mean_RGB, resnet_std_RGB),
])

### Create the train dataset object

In [ ]:
#https://stackoverflow.com/questions/49073799/pytorch-testing-with-torchvision-datasets-imagefolder-and-dataloader
train_dataset = torchvision.datasets.ImageFolder(
    root=TRAINDIR, transform=transform_train)

In [ ]:
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(resnet_mean_RGB, resnet_std_RGB),
])

In [ ]:
val_dataset = torchvision.datasets.ImageFolder(
    root=VALDIR, transform=transform_val)

### Create the train dataloader

In [ ]:
# fill this in
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, 
        sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

### Create the c

In [ ]:
# fill this in..
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [ ]:
best_acc1 = 0

In [ ]:
%%time
for epoch in range(START_EPOCH, EPOCHS):
    #See here for more: https://stackoverflow.com/questions/48324152/pytorch-how-to-change-the-learning-rate-of-an-optimizer-at-any-given-moment-no
    adjust_learning_rate(optimizer, epoch)
    
    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    #Adjusting by switching to manual scheduler (adjust_learning_rate)
    #scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    
    # 3. Log metrics over time to visualize performance
    wandb.log({'lr': scheduler.get_last_lr()[0]})